# RAG

Here we play with [this demo](https://github.com/LangGraph-GUI/LangGraph-learn/tree/main/07%20ollama%20RAG). We will build a Retrieval-Augmented Generation (RAG) system.

## Prerequesites

1. `ollama` server
2. `langchain-ollama`
3. `langchain-core`
4. `chromadb`: [Chroma](https://docs.trychroma.com/docs/overview/introduction) is a vector DB.

In [1]:
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb

In [2]:
llm_name = 'deepseek-r1:7b'

## Preprocessing External Files

We will work on one external markdown file for this demo.

In [3]:
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

In [4]:
file_path = 'files/elden_ring.md'
file_content = load_file_content(file_path)

First, we use a text splitter to slice the file content into text chunks.

In [5]:
# text splitter
# https://python.langchain.com/docs/how_to/recursive_text_splitter/
# https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html
# https://dev.to/eteimz/understanding-langchains-recursivecharactertextsplitter-2846
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)  # unit: number of chars

In [6]:
# split the document into chunks
chunks = text_splitter.split_text(file_content)

In [7]:
# debug sliced chunks
print([len(chunk) for chunk in chunks])
if len(chunks) >= 2:
    display(chunks[0][-100:])
    print('<SEPARATOR>')
    display(chunks[1][:100])

[365, 442, 488, 485, 388, 399, 435, 326, 316, 363, 285, 257, 296, 332, 414, 420, 477, 329, 261, 430, 392]


'gameplay FromSoftware is renowned for with a vast, open-world experience.\n\n---\n\n## Story and Setting'

<SEPARATOR>


'---\n\n## Story and Setting\n\n**Elden Ring** is set in the mystical world of the Lands Between, where t'

Then we use a embedding model to transform natural language text chunks into the embedding space. We will do that later.

In [8]:
# embedding model
# https://python.langchain.com/api_reference/ollama/embeddings/langchain_ollama.embeddings.OllamaEmbeddings.html
embed_model = OllamaEmbeddings(model=llm_name)

In [9]:
# debug embedding model
if len(chunks) > 0:
    embedded = embed_model.embed_documents([chunks[0]])
    print(len(embedded), len(embedded[0]))

1 3584


Now we create a ChromaDB client and initialize a collection to store the embedded chunks.

In [10]:
# Chroma Ephemeral Client (in-memory)
# https://docs.trychroma.com/docs/run-chroma/ephemeral-client
# https://docs.trychroma.com/reference/python/client#client
chroma_client = chromadb.Client()

In [11]:
# Chroma Collection
# https://docs.trychroma.com/docs/collections/create-get-delete
# https://docs.trychroma.com/reference/python/client#createcollection
collection = chroma_client.create_collection(name='docs')

In [12]:
# add embeded chunk to Chroma collection
# https://docs.trychroma.com/docs/collections/add-data
# https://docs.trychroma.com/reference/python/collection#add
for i, chunk in enumerate(chunks):
    embeded = embed_model.embed_documents([chunk])
    collection.add(
        ids=[str(i)],
        embeddings=embeded,  # note that embeded is already a list
        documents=[chunk]
    )

## Define the Execution Chain Including LLM

In [13]:
# initialize the ChatOllama model with desired parameters
# https://pypi.org/project/langchain-ollama/
# https://python.langchain.com/docs/integrations/chat/ollama/
# https://python.langchain.com/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html
llm = ChatOllama(model=llm_name)

In [14]:
# define a prompt template
template = """
I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [15]:
# define a langchain pipeline - execution chain
llm_chain = prompt | llm | StrOutputParser()

## Specify RAG Logic

Now we demonstrate how RAG system is actually implemented. We define the logic in one single function.

In [16]:
# debug embed_query: the embedded vector length should match the one in embed_documents
embedded_query = embed_model.embed_query('Describe the boss combat strategy of this game?')
print(len(embedded_query))

3584


In [17]:
# debug collection.query
# https://docs.trychroma.com/docs/querying-collections/query-and-get
# https://docs.trychroma.com/reference/python/collection#query
# how distances are calculated: https://docs.trychroma.com/docs/collections/configure
results = collection.query(
    query_embeddings=[embedded_query],
    n_results=3
)
print(results.keys())
print('ids:', results['ids'])
print('distances:', results['distances'])

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'distances', 'included'])
ids: [['8', '12', '0']]
distances: [[0.361456036567688, 0.38856959342956543, 0.4344329237937927]]


In [18]:
# answer questions using RAG
def answer(question: str) -> str:
    # embed query
    embedded_query = embed_model.embed_query(question)
    # retrieve the most relevant document chunks
    search_res = collection.query(
        query_embeddings=[embedded_query],
        n_results=3
    )
    search_documents = search_res['documents'][0]
    combined_docs = ''
    for i in range(len(search_documents)):
        combined_docs += f'Chunk {i}:\n{search_documents[i]}'
        if i != len(search_documents) - 1:
            combined_docs += '\n'
    # add RAG results to the initial question
    enhanced_question = f'Relevant text chunks retrieved from the knowledge base:\n{combined_docs}\nAnswer this question: {question}'
    formatted_prompt = prompt.format(question=enhanced_question)
    print(formatted_prompt)
    # execute
    generation = llm_chain.invoke(formatted_prompt)
    return generation

## Test

### Test 0

In [19]:
res = answer('What is Elden Ring?')


I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: Relevant text chunks retrieved from the knowledge base:
Chunk 0:
---

This comprehensive guide only scratches the surface of what Elden Ring has to offer. For more detailed walkthroughs, strategies, and tips, you can refer to the extensive resources available on platforms like [Fextralife](https://eldenring.wiki.fextralife.com/), [IGN](https://www.ign.com/wikis/elden-ring), and [TheGamer](https://www.thegamer.com/elden-ring-complete-guide-walkthrough/).
Chunk 1:
---

## Combat and Strategy

**Combat Mechanics:**
- **Basics**: Understanding the basics of combat, including dodging, parrying, and blocking, is crucial for survival.
- **Advanced Techniques**: Mastering advanced techniques, such as backstabbing and using magic effectively, can turn the tide in difficult battles.
Chunk 2:
---

## Character Bu

In [20]:
print(res)

<think>
Okay, so I need to answer the question "What is Elden Ring?" based on the provided knowledge base chunks. Let me go through each chunk step by step.

Chunk 0 says it's a comprehensive guide that only scratches the surface of what Elden Ring has to offer. It mentions resources like Fextralife, IGN, and TheGamer for more detailed info. I don't need to include this in my answer because I'm just answering "What is Elden Ring?".

Chunk 1 covers combat mechanics, which includes basics like dodging, parrying, blocking, backstabbing, and using magic effectively. This tells me that Elden Ring has various combat strategies beyond just attacking; it's probably a game with depth in its combat system.

Chunk 2 talks about character builds and playstyles under different categories: Strength, Dexterity, and Magic. Each build focuses on different aspects of gameplay—heavy weapons for strength, speed and precision for dexterity, and magic for distance attacks. This indicates that Elden Ring off

### Test 1

In [21]:
res = answer('Does Elden Ring support multiplayer mode?')


I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: Relevant text chunks retrieved from the knowledge base:
Chunk 0:
---

## Combat and Strategy

**Combat Mechanics:**
- **Basics**: Understanding the basics of combat, including dodging, parrying, and blocking, is crucial for survival.
- **Advanced Techniques**: Mastering advanced techniques, such as backstabbing and using magic effectively, can turn the tide in difficult battles.
Chunk 1:
---

This comprehensive guide only scratches the surface of what Elden Ring has to offer. For more detailed walkthroughs, strategies, and tips, you can refer to the extensive resources available on platforms like [Fextralife](https://eldenring.wiki.fextralife.com/), [IGN](https://www.ign.com/wikis/elden-ring), and [TheGamer](https://www.thegamer.com/elden-ring-complete-guide-walkthrough/).
Chunk 2:
---

## Character Bu

In [22]:
print(res)

<think>
Okay, so I'm trying to figure out if Elden Ring supports multiplayer mode based on the provided knowledge base. Let me go through the text chunks step by step.

First, looking at Chunk 0, it's about combat and strategy with basics like dodging, parrying, blocking, advanced techniques such as backstabbing and magic use. No mention of multiplayer here.

Chunk 1 says this guide is just a scratch since there are extensive resources elsewhere for more detailed info. The links provided include Fextralife, IGN, and TheGamer, but none talk about multiplayer support in Elden Ring.

Then Chunk 2 covers character builds with strengths like strength, dexterity, magic builds, but again nothing on multiplayer aspects or features related to playing with others.

So from all these chunks, there's no information provided about whether Elden Ring has a multiplayer mode. They either don't mention it or refer to external sites that probably also don't discuss multiplayer features.
</think>

Based 

### Test 2

In [23]:
res = answer('Who produced the soundtrack for Elden Ring?')


I am helping the user to maintain a knowledge base - some local files.
I will answer the user's questions strictly based on the content retrieved from the local files.
Current question: Relevant text chunks retrieved from the knowledge base:
Chunk 0:
---

This comprehensive guide only scratches the surface of what Elden Ring has to offer. For more detailed walkthroughs, strategies, and tips, you can refer to the extensive resources available on platforms like [Fextralife](https://eldenring.wiki.fextralife.com/), [IGN](https://www.ign.com/wikis/elden-ring), and [TheGamer](https://www.thegamer.com/elden-ring-complete-guide-walkthrough/).
Chunk 1:
---

## Combat and Strategy

**Combat Mechanics:**
- **Basics**: Understanding the basics of combat, including dodging, parrying, and blocking, is crucial for survival.
- **Advanced Techniques**: Mastering advanced techniques, such as backstabbing and using magic effectively, can turn the tide in difficult battles.
Chunk 2:
The world is designe

In [24]:
print(res)

<think>
Okay, so I need to figure out who produced the soundtrack for Elden Ring. Let me start by recalling what I know about the game. Elden Ring is a pretty popular action RPG with a lot of content and various resources available online, like guides and walkthroughs. The user mentioned that they're maintaining a knowledge base using some local files, which probably include this text chunk from the guide.

Looking at the provided chunks, there's one labeled as Chunk 0, which talks about guides and resources on external sites like Fextralife, IGN, and TheGamer. Then Chunk 2 mentions that the world rewards exploration with hidden secrets, powerful items, and lore fragments, but doesn't directly answer any specific questions.

The question to answer is: "Who produced the soundtrack for Elden Ring?" So I need to find out who did the music for this game. I'm not exactly an expert in video game history, but I can try to piece it together based on what I remember or have heard before.

I kno